Mini Reco

Defines

In [1]:
import numpy as np

testcase = "000.txt"
file = open("./data/mini_reco/input/input"+testcase, "r")
inputs = file.readlines()
file.close()

num_sim_user_top_N = int(inputs[0])
print('num_sim_user_top_N:', num_sim_user_top_N)

num_item_rec_top_M = int(inputs[1])
print('num_item_rec_top_M:', num_item_rec_top_M)

num_users = int(inputs[2])
print('num_users:', num_users)

num_items = int(inputs[3])
print('num_items:', num_items)

num_rows = int(inputs[4])
print('num_rows:', num_rows)
print('='*80)

ratings = np.array([rating.split() for rating in inputs[5:num_rows+5]])
print('ratings:', ratings.shape)
print(ratings)
print('='*80)

num_reco_users = int(inputs[num_rows+5])
print('num_reco_users:', num_reco_users)
print('='*80)

who = np.array([u for u in inputs[num_rows+6:]]).astype(int) - 1
print('who:', who.shape)
print(who+1)

U = np.array([np.zeros(num_sim_user_top_N) for _ in range(num_users)]).astype(int)
I_hat = np.array([np.zeros(num_item_rec_top_M) for _ in range(num_users)])

rating_matrix = np.array([np.zeros(num_items) for _ in range(num_users)])

num_sim_user_top_N: 2
num_item_rec_top_M: 2
num_users: 5
num_items: 10
num_rows: 15
ratings: (15, 3)
[['1' '1' '1.0']
 ['1' '2' '2.0']
 ['1' '5' '1.2']
 ['2' '2' '1.5']
 ['2' '3' '3.0']
 ['3' '1' '2.2']
 ['3' '2' '6.2']
 ['3' '7' '1.5']
 ['4' '6' '1.2']
 ['4' '3' '1.5']
 ['4' '1' '3.1']
 ['4' '2' '4.0']
 ['5' '4' '8.2']
 ['5' '2' '6.5']
 ['5' '7' '8.0']]
num_reco_users: 2
who: (2,)
[1 2]


In [2]:
for r in ratings:
    rating_matrix[int(r[0])-1, int(r[1])-1] = float(r[2])

print('rating matrix:', rating_matrix.shape)
print(rating_matrix)

rating matrix: (5, 10)
[[1.  2.  0.  0.  1.2 0.  0.  0.  0.  0. ]
 [0.  1.5 3.  0.  0.  0.  0.  0.  0.  0. ]
 [2.2 6.2 0.  0.  0.  0.  1.5 0.  0.  0. ]
 [3.1 4.  1.5 0.  0.  1.2 0.  0.  0.  0. ]
 [0.  6.5 0.  8.2 0.  0.  8.  0.  0.  0. ]]


In [4]:
similarities = np.array([np.zeros(num_users) for _ in range(num_users)])

user_vectors = rating_matrix #U_svd rating_matrix

for i in range(num_users):
    similarities[i] = np.array([np.dot(user_vectors[i], u)/(np.linalg.norm(user_vectors[i]) * np.linalg.norm(u)) for u in user_vectors])
    similarities[i][i] = 1.0 + 1e-100
    
    U[i] = np.flip(similarities[i].argsort())[1:num_sim_user_top_N + 1]
    if i == 50:
        print(U[i])

print('similarities:', similarities.shape)
print(similarities)
print('='*80)
print('U:', U.shape)
print(U+1)

similarities: (5, 5)
[[1.         0.35245369 0.85263096 0.80806489 0.38892261]
 [0.35245369 1.         0.41092055 0.57833467 0.22069431]
 [0.85263096 0.41092055 1.         0.86572355 0.58845823]
 [0.80806489 0.57833467 0.86572355 1.         0.36467191]
 [0.38892261 0.22069431 0.58845823 0.36467191 1.        ]]
U: (5, 2)
[[3 4]
 [4 3]
 [4 1]
 [3 1]
 [3 1]]


In [5]:
rating_matrix_hat = np.array([np.zeros(num_items) for _ in range(num_users)])

for i in range(num_users):
    for u in U[i]:
        r_hat = (rating_matrix[u] - (np.sum(rating_matrix[u])/np.sum(rating_matrix[u]!=0))) / (np.std(rating_matrix[u][rating_matrix[u]!=0]) + 1e-100)
        rating_matrix_hat[i] += r_hat  * similarities[i, u]
        
    rating_matrix_hat[i] /= np.sum(np.absolute(similarities[i, U[i]]))

    bias = np.sum(rating_matrix[i])/np.sum(rating_matrix[i]!=0)

    rating_matrix_hat[i] += bias

print('rating_matrix_hat:', rating_matrix_hat.shape)
print(rating_matrix_hat)

rating_matrix_hat: (5, 10)
[[ 1.40225013  2.77496093  0.17971726 -0.45495506 -0.45495506  0.05278279
  -0.08298978 -0.45495506 -0.45495506 -0.45495506]
 [ 2.35974543  3.61977969  1.10498663  0.3424437   0.3424437   0.95247804
   0.6433844   0.3424437   0.3424437   0.3424437 ]
 [ 3.12537895  4.66812229  1.27596887  0.61882588  1.99697493  1.14454027
   0.61882588  0.61882588  0.61882588  0.61882588]
 [ 1.72823959  3.84490959  0.06123689  0.06123689  1.40212807  0.06123689
   0.43595965  0.06123689  0.06123689  0.06123689]
 [ 6.87838307  8.96259002  5.31761319  5.31761319  6.42282936  5.31761319
   5.75381074  5.31761319  5.31761319  5.31761319]]


In [6]:
results = []
for i in who:
    reco_items = np.flip(rating_matrix_hat[i].argsort())
    rating = rating_matrix[i]!= 0
    result = np.array([r for r in reco_items if rating[r] == False][:num_item_rec_top_M]) + 1
    results.append(result)

results = np.array(results)

print('results:', results.shape)
print(results)

results: (2, 2)
[[3 6]
 [1 6]]


In [7]:
import math

def ndcg(gt, rec):
    idcg = sum([1.0 / math.log(i + 2, 2) for i in range(len(gt))])
    dcg = 0.0
    for i, r in enumerate(rec):
        if r not in gt:
            continue
        gt_index = gt.index(r)
        if i != gt_index:
            rel = 0.7
        else:
            rel = 1.0
        dcg += rel / math.log(i + 2, 2)
    ndcg = dcg / idcg

    return ndcg

In [8]:
file = open("./data/mini_reco/output/output"+testcase, "r")
test_inputs = file.readlines()
file.close()

gt = np.array([item.split() for item in test_inputs]).astype(int)
print('gt:', gt.shape)
print(gt)
print('='*80)

print(ndcg(gt.tolist(), results.tolist()))

gt: (2, 2)
[[3 6]
 [1 6]]
1.0
